## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [ ]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [ ]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_32.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_32.csv')

In [ ]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [ ]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [ ]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [ ]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [ ]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 4000)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.2)
        , 'max_depth': trial.suggest_int('max_depth', 3, 15)

        , 'alpha': trial.suggest_float('alpha', 0.00001, 0.01, log=True)
        , 'gamma': trial.suggest_float('gamma', 0.00001, 0.01, log=True)

        , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.3, 1)
        , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.5, 1)
        
        , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1)
        , 'subsample' : trial.suggest_float('subsample', 0.1, 1)
        , 'objective': 'binary:logistic'  # 이진 분류
        , 'tree_method' : "exact"        # 트리 메소드

        , 'random_state': 42
        , 'metric': 'auc'
        , 'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-16 13:42:31,039] A new study created in memory with name: no-name-7426a5be-e245-40ee-a6ff-e6301d27fc06
[I 2025-02-16 13:42:40,559] Trial 0 finished with value: 0.7342956504325373 and parameters: {'n_estimators': 2452, 'num_leaves': 646, 'max_depth': 295, 'learning_rate': 0.1506310150441879, 'min_child_samples': 4, 'subsample': 0.6667793661211553, 'colsample_bytree': 0.9777300787314716, 'reg_alpha': 9.781614248096584, 'reg_lambda': 3.4821211122603057}. Best is trial 0 with value: 0.7342956504325373.
[I 2025-02-16 13:42:50,239] Trial 1 finished with value: 0.73483596994366 and parameters: {'n_estimators': 1585, 'num_leaves': 2239, 'max_depth': 139, 'learning_rate': 0.19393610653218132, 'min_child_samples': 142, 'subsample': 0.764845699246905, 'colsample_bytree': 0.5975600135919459, 'reg_alpha': 9.30764429632497, 'reg_lambda': 8.3885111738792}. Best is trial 1 with value: 0.73483596994366.
[I 2025-02-16 13:43:07,652] Trial 2 finished with value: 0.736850177797556 and parameters

.